# Modelo de Categorización

In [ ]:
# Standard library imports
import os
import sys
import re
import math
import json
import logging
from datetime import datetime
from typing import Dict, List, Tuple, Union
from collections import Counter

# Third-party imports
import pandas as pd
import numpy as np
import swifter # type: ignore
from unidecode import unidecode # type: ignore
from tqdm.auto import tqdm # type: ignore

# Plotly for visualization
import plotly.express as px # type: ignore
import plotly.graph_objects as go # type: ignore
from plotly.subplots import make_subplots # type: ignore

# --- Configuración del Logging ---
# Se establece un sistema de logging para registrar información, advertencias y errores durante la ejecución.
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class EnhancedSentimentAnalyzer:
    """
    Analizador avanzado de sentimiento y categorización de texto.

    Esta clase implementa una lógica sofisticada para procesar comentarios de texto,
    asignarles categorías temáticas y calcular una puntuación de sentimiento detallada.

    Características principales:
    - Manejo contextual de negaciones e intensificadores: El sentimiento no solo depende
      de las palabras individuales, sino también de las que las rodean (e.g., "no muy bueno").
    - Modelo híbrido NPS + análisis semántico: Combina la puntuación numérica de NPS
      (Net Promoter Score) con el análisis del texto para un resultado de sentimiento más preciso.
    - Categorización jerárquica: Asigna una categoría principal y una secundaria basada en
      palabras clave y umbrales dinámicos.
    - Conteo de palabras de sentimiento mejorado: Evita contar la misma palabra de sentimiento
      múltiples veces para obtener un recuento único.
    """

    def __init__(self, config: Dict):
        """
        Inicializa el analizador con una configuración específica.

        Args:
            config (Dict): Un diccionario que contiene todos los parámetros y recursos
                          lingüísticos necesarios para el análisis, como listas de palabras,
                          pesos, umbrales y reglas.
        """
        self.config = self._validate_and_complete_config(config)
        self._initialize_linguistic_resources()

    def _validate_and_complete_config(self, config: Dict) -> Dict:
        """
        Valida que la configuración cargada sea completa y tenga el formato correcto.

        Este método asegura que todas las claves y sub-claves necesarias para la operación
        estén presentes en el diccionario de configuración, previniendo errores en tiempo
        de ejecución.

        Args:
            config (Dict): El diccionario de configuración a validar.

        Returns:
            Dict: El diccionario de configuración validado.

        Raises:
            ValueError: Si falta una clave de configuración requerida.
            TypeError: Si una clave de configuración tiene un tipo de dato incorrecto.
        """
        # Define las claves de primer nivel y sus tipos esperados.
        required_top_level_keys = {
            'categorias': dict,
            'pesos_categorias': dict,
            'umbral_secundario': (int, float),
            'sentimiento_params': dict
        }
        for key, key_type in required_top_level_keys.items():
            if key not in config:
                raise ValueError(f"Clave de configuración de nivel superior faltante: '{key}'")
            if not isinstance(config[key], key_type):
                raise TypeError(f"Tipo incorrecto para '{key}'. Esperado: {key_type}")

        # Valida que todas las claves necesarias dentro de 'sentimiento_params' existan.
        required_sentiment_keys = [
            'base_scores', 'factor_pos', 'factor_neg', 'umbral_detractor',
            'umbral_promotor', 'context_window', 'intensifier_weights',
            'special_phrases', 'negation_impact', 'lemmatization_map',
            'stopwords', 'negation_words'
        ]
        for key in required_sentiment_keys:
            if key not in config['sentimiento_params']:
                raise ValueError(f"Clave faltante en 'sentimiento_params' del archivo JSON: '{key}'")
        
        return config

    def _initialize_linguistic_resources(self):
        """
        Inicializa y pre-procesa los recursos lingüísticos desde la configuración.

        Carga listas de palabras (negaciones, intensificadores, stopwords, sentimiento)
        y las convierte a sets para búsquedas más eficientes. También realiza ajustes,
        como eliminar negaciones e intensificadores de las stopwords para que no sean
        ignorados durante el análisis de sentimiento.
        """
        sentiment_params = self.config['sentimiento_params']

        # Cargar recursos directamente desde la configuración.
        self.negation_words = set(sentiment_params['negation_words'])
        self.intensifiers_map = sentiment_params['intensifier_weights']
        self.intensifiers = set(self.intensifiers_map.keys())
        self.stopwords = set(self.config['stopwords'])
        self.special_phrases = sentiment_params['special_phrases']
        self.lemmatization_map = sentiment_params['lemmatization_map']
        
        # Asegura que las palabras de negación e intensificadores no sean eliminadas
        # como stopwords, ya que son cruciales para el análisis de contexto.
        self.stopwords.difference_update(self.negation_words)
        self.stopwords.difference_update(self.intensifiers)

        # Prepara patrones de regex y listas de palabras de sentimiento lematizadas.
        self.punct_pattern = re.compile(r'[!\"#$%&\'()*+,-./:;<=>?@\[\\\]^_`{|}~]')
        self.pos_words_original = set(self.config['categorias'].get('sentimiento_positivo', []))
        self.neg_words_original = set(self.config['categorias'].get('sentimiento_negativo', []))
        # Lematiza las palabras de sentimiento para una coincidencia más robusta.
        self.pos_words = {self._lemmatize_word(w) for w in self.pos_words_original}
        self.neg_words = {self._lemmatize_word(w) for w in self.neg_words_original}

    def _lemmatize_word(self, word: str) -> str:
        """
        Lematiza una palabra usando el mapa de lematización proporcionado.

        Si la palabra no está en el mapa, la devuelve sin cambios. La lematización
        reduce las palabras a su forma raíz (e.g., "buenos" -> "bueno").

        Args:
            word (str): La palabra a lematizar.

        Returns:
            str: La palabra lematizada o la original si no hay mapeo.
        """
        return self.lemmatization_map.get(word, word)

    def limpiar_texto(self, texto: str) -> str:
        """
        Realiza una limpieza completa del texto de entrada.

        El proceso de limpieza incluye:
        1. Manejo de valores nulos (NaN).
        2. Conversión a minúsculas y eliminación de acentos (unidecode).
        3. Eliminación de signos de puntuación.
        4. Normalización de espacios en blanco.
        5. Lematización de cada palabra y eliminación de stopwords.

        Args:
            texto (str): El texto a limpiar.

        Returns:
            str: El texto procesado y listo para el análisis.
        """
        if pd.isna(texto):
            return ""
        texto = unidecode(str(texto).lower()) # Normaliza y convierte a minúsculas
        texto = self.punct_pattern.sub(' ', texto) # Elimina puntuación
        texto = re.sub(r'\s+', ' ', texto).strip() # Normaliza espacios
        # Lematiza y filtra stopwords y palabras cortas
        return ' '.join([self._lemmatize_word(w) for w in texto.split() if self._lemmatize_word(w) not in self.stopwords and len(w) > 1])

    def _tokenizar_texto(self, texto: str, n_gram_range: Tuple[int, int] = (1, 3)) -> List[str]:
        """
        Convierte un texto en una lista de tokens (n-gramas).

        Un n-grama es una secuencia de n palabras. Esta función genera n-gramas de
        diferentes tamaños (e.g., unigramas, bigramas, trigramas) para capturar
        frases y no solo palabras individuales.

        Args:
            texto (str): El texto a tokenizar.
            n_gram_range (Tuple[int, int], optional): Rango de tamaños de n-gramas.
                                                     Defaults to (1, 3).

        Returns:
            List[str]: Una lista de todos los tokens (n-gramas) generados.
        """
        words = texto.split()
        lemmatized_words = [self._lemmatize_word(word) for word in words]
        all_tokens = []
        min_n, max_n = n_gram_range
        # Genera n-gramas para cada 'n' en el rango especificado
        for n in range(min_n, max_n + 1):
            for i in range(len(lemmatized_words) - n + 1):
                all_tokens.append(' '.join(lemmatized_words[i:i+n]))
        return all_tokens

    def _calcular_puntuaciones_categoria(self, tokens: List[str]) -> Dict[str, float]:
        """
        Calcula una puntuación para cada categoría temática basada en los tokens.

        Compara los tokens del texto con las listas de palabras clave de cada categoría.
        La puntuación se basa en la cantidad de coincidencias, ponderada por el peso
        de la categoría y un factor de ajuste.

        Args:
            tokens (List[str]): La lista de tokens del texto.

        Returns:
            Dict[str, float]: Un diccionario con las puntuaciones para cada categoría.
        """
        token_set = set(tokens) # Usar un set mejora la eficiencia de la búsqueda.
        puntuaciones = {}
        for categoria, keywords_list in self.config['categorias'].items():
            if categoria.startswith('sentimiento_'): # Ignora las listas de sentimiento
                continue
            
            keywords_set = {self._lemmatize_word(kw) for kw in keywords_list}
            palabras_encontradas = token_set.intersection(keywords_set)
            score = len(palabras_encontradas)
            
            if score > 0:
                # Aplica el peso específico de la categoría
                peso_categoria = self.config['pesos_categorias'].get(categoria, 1.0)
                # Aplica un ajuste para no favorecer excesivamente a categorías con miles de keywords
                ajuste_por_num_keywords = (1 - 0.2 * len(keywords_set) / 1000) if len(keywords_set) > 0 else 1
                puntuaciones[categoria] = score * peso_categoria * ajuste_por_num_keywords
        return puntuaciones

    def categorizar(self, texto: str) -> Tuple[str, str]:
        """
        Asigna una categoría principal y secundaria a un texto.

        Orquesta el proceso de limpieza, tokenización y cálculo de puntuaciones para
        determinar las categorías más relevantes. Si no se encuentra ninguna categoría,
        devuelve 'no_clasificado'.

        Args:
            texto (str): El texto a categorizar.

        Returns:
            Tuple[str, str]: Una tupla conteniendo la categoría principal y la secundaria.
        """
        # Maneja casos de texto vacío o sin respuesta.
        if pd.isna(texto) or not str(texto).strip() or str(texto).strip().lower() == 'sin_respuesta':
            return ("sin_respuesta", "sin_respuesta")

        texto_limpio = self.limpiar_texto(texto)
        if not texto_limpio or 'sinrespuesta' in texto_limpio.split():
            return ("sin_respuesta", "sin_respuesta")

        # Genera tokens (unigramas y bigramas) para la categorización.
        tokens_categorizacion = self._tokenizar_texto(texto_limpio, n_gram_range=(1, 2))
        
        # Filtra categorías con puntuaciones por debajo de un umbral mínimo.
        umbral_score_categoria = self.config.get('umbral_score_categoria', 0.5)
        puntuaciones = {k: v for k, v in self._calcular_puntuaciones_categoria(tokens_categorizacion).items() if v > umbral_score_categoria}

        if not puntuaciones:
            return ('no_clasificado', 'no_clasificado')

        # Normaliza las puntuaciones y las ordena para encontrar la principal y secundaria.
        max_score = max(puntuaciones.values())
        normalized = {k: v / max_score if max_score > 0 else 0 for k, v in puntuaciones.items()}
        sorted_cats = sorted(normalized.items(), key=lambda x: (-x[1], x[0]))
        
        principal, p_score = sorted_cats[0]
        secundaria = principal # Por defecto, la secundaria es la misma que la principal.
        
        # Si hay más de una categoría, evalúa si la segunda es lo suficientemente relevante.
        if len(sorted_cats) > 1:
            next_cat_name, next_score = sorted_cats[1]
            umbral_relativo = self.config['umbral_secundario']
            # La segunda categoría se asigna si su puntuación es al menos un % de la principal.
            if next_score >= umbral_relativo * p_score and next_cat_name != principal:
                secundaria = next_cat_name
        return (principal, secundaria)

    def _contar_sentimientos_avanzado(self, texto_limpio_con_contexto: str) -> Tuple[float, float, int, int]:
        """
        Calcula las puntuaciones de sentimiento positivo y negativo de forma contextual.

        Este es el núcleo del análisis de sentimiento. Primero busca frases especiales
        predefinidas y luego analiza palabras individuales, considerando en ambos casos:
        - Negaciones (e.g., "no bueno").
        - Intensificadores (e.g., "muy bueno").
        - Una "ventana de contexto" alrededor de cada palabra/frase.

        Args:
            texto_limpio_con_contexto (str): El texto limpio (lematizado pero con stopwords
                                             relevantes como negaciones e intensificadores).

        Returns:
            Tuple[float, float, int, int]: Una tupla con (puntuación_positiva,
                                             puntuación_negativa, conteo_palabras_pos_unicas,
                                             conteo_palabras_neg_unicas).
        """
        c_pos_score = 0.0
        c_neg_score = 0.0
        params = self.config['sentimiento_params']
        window_size = params['context_window']
        negation_impact = params['negation_impact']
        tokens_del_texto_para_analisis = texto_limpio_con_contexto.split()
        indices_usados_en_frases = [False] * len(tokens_del_texto_para_analisis)
        
        # Ordena las frases especiales de más largas a más cortas para evitar coincidencias parciales.
        sorted_special_phrases_items = sorted(
            self.special_phrases.items(),
            key=lambda item: len(self._tokenizar_texto(item[0], n_gram_range=(1,1))),
            reverse=True
        )

        # 1. Búsqueda de frases especiales (e.g., "muy buen servicio").
        for phrase_str, sentiment_info in sorted_special_phrases_items:
            phrase_tokens_lematizados = [self._lemmatize_word(w) for w in phrase_str.split()]
            phrase_len_n = len(phrase_tokens_lematizados)
            if phrase_len_n == 0: continue
            for i in range(len(tokens_del_texto_para_analisis) - phrase_len_n + 1):
                if any(indices_usados_en_frases[k] for k in range(i, i + phrase_len_n)):
                    continue # Salta si los tokens ya fueron usados en otra frase.
                phrase_candidate_tokens_lematizados = [self._lemmatize_word(tok) for tok in tokens_del_texto_para_analisis[i:i+phrase_len_n]]
                if phrase_candidate_tokens_lematizados == phrase_tokens_lematizados:
                    base_value = sentiment_info['weight']
                    context_before = tokens_del_texto_para_analisis[max(0, i - window_size):i]
                    context_after = tokens_del_texto_para_analisis[i + phrase_len_n : min(len(tokens_del_texto_para_analisis), i + phrase_len_n + window_size)]
                    context_around_phrase_tokens = context_before + context_after
                    # Verifica si hay una negación en el contexto de la frase.
                    is_negated_special = any(self._lemmatize_word(tok) in self.negation_words for tok in context_around_phrase_tokens)
                    score_modifier = -negation_impact if is_negated_special else 1
                    
                    if sentiment_info['sentiment'] == 'pos':
                        c_pos_score += base_value * score_modifier
                    elif sentiment_info['sentiment'] == 'neg':
                        c_neg_score += base_value * score_modifier
                    
                    # Marca los tokens como usados para no volver a contarlos.
                    for k_idx in range(i, i + phrase_len_n):
                        indices_usados_en_frases[k_idx] = True
        
        # 2. Búsqueda de palabras de sentimiento individuales.
        unique_positive_words_found = set()
        unique_negative_words_found = set()
        for i, token_original in enumerate(tokens_del_texto_para_analisis):
            if indices_usados_en_frases[i]:
                continue # Salta si el token ya formó parte de una frase.
            
            lemmatized_token = self._lemmatize_word(token_original)
            context_tokens_before = tokens_del_texto_para_analisis[max(0, i - window_size):i]
            context_tokens_after = tokens_del_texto_para_analisis[i+1 : min(len(tokens_del_texto_para_analisis), i + 1 + window_size)]
            context_tokens_for_word = context_tokens_before + context_tokens_after
            
            # Calcula el bono de intensidad de las palabras en el contexto.
            intensity_bonus = sum(self.intensifiers_map.get(self._lemmatize_word(tok), 0) for tok in context_tokens_for_word)
            effective_intensity = 1 + intensity_bonus
            
            # Verifica si hay negación en el contexto.
            is_negated = any(self._lemmatize_word(tok) in self.negation_words for tok in context_tokens_for_word)
            score_effect = effective_intensity * (-negation_impact if is_negated else 1)
            
            if lemmatized_token in self.pos_words:
                c_pos_score += score_effect
                unique_positive_words_found.add(lemmatized_token)
            elif lemmatized_token in self.neg_words:
                c_neg_score += score_effect
                unique_negative_words_found.add(lemmatized_token)
                
        return round(c_pos_score, 2), round(c_neg_score, 2), len(unique_positive_words_found), len(unique_negative_words_found)

    def analizar_sentimiento(self, texto_original: str, prob_recomendar: Union[int, float, None]) -> Dict[str, Union[str, float, int]]:
        """
        Realiza el análisis de sentimiento completo, combinando NPS y texto.

        Calcula una puntuación final de sentimiento partiendo de una base dada por la
        puntuación NPS y ajustándola con los resultados del análisis de texto.

        Args:
            texto_original (str): El comentario original del usuario.
            prob_recomendar (Union[int, float, None]): La puntuación NPS (0-10).

        Returns:
            Dict[str, Union[str, float, int]]: Un diccionario con los resultados detallados
                                              del análisis de sentimiento.
        """
        # Normaliza la puntuación NPS.
        try:
            prob = 7 if pd.isna(prob_recomendar) else max(0, min(10, int(float(prob_recomendar))))
        except (TypeError, ValueError):
            prob = 7 # Valor neutral por defecto si hay un error.

        # Lógica para respuestas sin comentario. El sentimiento se basa solo en el NPS.
        if pd.isna(texto_original) or not str(texto_original).strip() or str(texto_original).strip().lower() == 'sin_respuesta':
            segmento_nps_calc = 'promotor' if prob >= 9 else 'detractor' if prob <= 6 else 'neutro'
            base_score = self.config['sentimiento_params']['base_scores'].get(segmento_nps_calc, 0)
            sentiment_final_override = self._determinar_sentimiento(base_score, prob)
            return {
                'segmento_nps': segmento_nps_calc,
                'sentiment_override': sentiment_final_override,
                'sentiment_libre': 'neutral',
                'puntuacion': base_score,
                'palabras_positivas_score': 0.0,
                'palabras_negativas_score': 0.0,
                'conteo_palabras_positivas': 0,
                'conteo_palabras_negativas': 0
            }

        # Lógica para respuestas con comentario.
        texto_limpio_global = self.limpiar_texto(texto_original)
        if not texto_limpio_global:
            # Si el texto queda vacío tras la limpieza, se trata como si no hubiera comentario.
            return self.analizar_sentimiento(None, prob)

        c_pos_score, c_neg_score, count_pos_words, count_neg_words = self._contar_sentimientos_avanzado(texto_limpio_global)
        
        params = self.config['sentimiento_params']
        # Obtiene la puntuación base del NPS.
        base_score_nps = params['base_scores'].get('promotor' if prob >= 9 else 'detractor' if prob <= 6 else 'neutro', 0)
        # Calcula la puntuación final combinando la base NPS con el análisis de texto.
        puntuacion_final_sentimiento = base_score_nps + (params['factor_pos'] * c_pos_score) - (params['factor_neg'] * c_neg_score)
        
        return {
            'segmento_nps': 'promotor' if prob >= 9 else 'detractor' if prob <= 6 else 'neutro',
            'sentiment_override': self._determinar_sentimiento(puntuacion_final_sentimiento, prob),
            'sentiment_libre': self._determinar_sentimiento(puntuacion_final_sentimiento),
            'puntuacion': round(puntuacion_final_sentimiento, 3),
            'palabras_positivas_score': c_pos_score,
            'palabras_negativas_score': c_neg_score,
            'conteo_palabras_positivas': count_pos_words,
            'conteo_palabras_negativas': count_neg_words
        }

    def _determinar_sentimiento(self, puntuacion: float, prob: Union[int, None] = None) -> str:
        """
        Asigna una etiqueta de sentimiento final ('positivo', 'negativo', 'neutral').

        Utiliza umbrales para clasificar la puntuación. Si se proporciona la puntuación NPS,
        aplica reglas de "flexibilización" para casos límite (e.g., un promotor con
        texto ligeramente negativo puede seguir siendo 'positivo' o 'neutral').

        Args:
            puntuacion (float): La puntuación final de sentimiento.
            prob (Union[int, None], optional): La puntuación NPS. Defaults to None.

        Returns:
            str: La etiqueta de sentimiento.
        """
        params = self.config['sentimiento_params']
        umbral_detractor = params['umbral_detractor']
        umbral_promotor = params['umbral_promotor']
        promoter_negative_text_leniency = params.get('promoter_negative_text_leniency', -0.5)
        detractor_positive_text_leniency = params.get('detractor_positive_text_leniency', 0.5)
        
        # Reglas especiales cuando se conoce el NPS.
        if prob is not None:
            if prob == 10: return 'positivo' # Un 10 es siempre positivo.
            if prob <= 2: return 'negativo'  # Un 0-2 es siempre negativo.
            if prob == 9: # Un promotor 9 puede ser neutral si el texto es muy negativo.
                return 'neutral' if puntuacion < umbral_detractor + promoter_negative_text_leniency else 'positivo'
            if prob in [3, 4]: # Un detractor bajo puede ser neutral si el texto es muy positivo.
                return 'neutral' if puntuacion > umbral_promotor - detractor_positive_text_leniency else 'negativo'

        # Reglas generales basadas solo en la puntuación.
        if puntuacion <= umbral_detractor: return 'negativo'
        if puntuacion >= umbral_promotor: return 'positivo'
        return 'neutral'

def cargar_config_mejorada(ruta_categorias: str, ruta_sentimientos: str) -> Dict:
    """
    Carga y combina los archivos de configuración de categorías y sentimientos.

    Lee dos archivos JSON separados, uno para las categorías temáticas y otro para los
    parámetros de sentimiento, y los fusiona en un único diccionario de configuración
    para el analizador.

    Args:
        ruta_categorias (str): Ruta al archivo JSON de categorías.
        ruta_sentimientos (str): Ruta al archivo JSON de sentimientos.

    Returns:
        Dict: El diccionario de configuración combinado.
    
    Raises:
        FileNotFoundError: Si alguno de los archivos JSON no se encuentra.
        json.JSONDecodeError: Si hay un error de formato en alguno de los JSON.
    """
    try:
        with open(ruta_categorias, 'r', encoding='utf-8') as f:
            categorias_config = json.load(f)
        with open(ruta_sentimientos, 'r', encoding='utf-8') as f:
            sentimientos_config = json.load(f)

        # Combina la información de ambos archivos en una estructura única.
        config = {
            "categorias": {**categorias_config.get('categorias', {})},
            "pesos_categorias": categorias_config.get('pesos_categorias', {}),
            "umbral_secundario": categorias_config.get('umbral_secundario', 0.7),
            "umbral_score_categoria": categorias_config.get('umbral_score_categoria', 0.5),
            "sentimiento_params": sentimientos_config.get('sentimiento_params', {}),
            'stopwords': sentimientos_config.get('sentimiento_params', {}).get('stopwords', [])
        }
        # Añade las listas de palabras de sentimiento al diccionario de categorías para un acceso unificado.
        config['categorias']['sentimiento_positivo'] = sentimientos_config.get('sentimiento_positivo', [])
        config['categorias']['sentimiento_negativo'] = sentimientos_config.get('sentimiento_negativo', [])
        return config
    except FileNotFoundError as e:
        logger.error(f"Error: Archivo de configuración no encontrado: {e.filename}")
        raise
    except json.JSONDecodeError as e:
        logger.error(f"Error decodificando JSON: {e.msg} en {e.doc}")
        raise

def procesar_dataframe_eficiente(df: pd.DataFrame, config: Dict) -> pd.DataFrame:
    """
    Procesa un DataFrame completo aplicando una estrategia de categorización en dos pasadas.

    Esta función está optimizada para reducir el número de comentarios 'no_clasificado'.
    1.  **Primera pasada**: Categoriza todos los comentarios usando todas las categorías
        excepto una categoría general de "satisfacción" ('Extras_Satisfaccion'). Esto
        fuerza la asignación de categorías más específicas primero.
    2.  **Segunda pasada**: Toma los comentarios que quedaron como 'no_clasificado' en la
        primera pasada y intenta categorizarlos usando únicamente la categoría de
        "satisfacción". Esto captura comentarios positivos genéricos sin sobrescribir
        categorías más específicas.
    3.  **Análisis de Sentimiento**: Se ejecuta al final sobre todos los comentarios.

    Args:
        df (pd.DataFrame): El DataFrame de entrada.
        config (Dict): El diccionario de configuración completo.

    Returns:
        pd.DataFrame: El DataFrame procesado con las nuevas columnas de análisis.
    """
    if not isinstance(df, pd.DataFrame):
        raise TypeError("La entrada debe ser un DataFrame de Pandas.")

    # --- INICIO DE LA ESTRATEGIA DE DOS PASADAS ---

    # Copia la configuración para poder modificarla sin afectar el original.
    config_primera_pasada = config.copy()
    config_primera_pasada['categorias'] = config['categorias'].copy()
    config_primera_pasada['pesos_categorias'] = config['pesos_categorias'].copy()
    
    # Paso 1: Preparar la configuración para la primera pasada.
    # Se extrae temporalmente la categoría de satisfacción general.
    nombre_categoria_extra = 'Extras_Satisfaccion'
    satisfaction_keywords = config_primera_pasada['categorias'].pop(nombre_categoria_extra, None)
    satisfaction_weight = config_primera_pasada['pesos_categorias'].pop(nombre_categoria_extra, None)
    
    # Instancia el analizador para la primera pasada (sin la categoría de satisfacción).
    analyzer_primera_pasada = EnhancedSentimentAnalyzer(config_primera_pasada)
    
    logger.info("Iniciando el pre-procesamiento de texto...")
    df['razon_recomendar_limpio'] = df['razon_recomendar'].swifter.apply(analyzer_primera_pasada.limpiar_texto)

    # Separa el DataFrame en registros con y sin comentarios para un procesamiento más eficiente.
    mask_sin_comentarios = df['razon_recomendar_limpio'].isna() | (df['razon_recomendar_limpio'] == '')
    df_con_comentarios = df[~mask_sin_comentarios].copy()
    df_sin_comentarios = df[mask_sin_comentarios].copy()

    if not df_con_comentarios.empty:
        logger.info(f"Procesando {len(df_con_comentarios)} registros con comentarios...")

        # Paso 2: Ejecutar la primera pasada de categorización.
        logger.info("Ejecutando primera pasada de categorización (sin categoría de satisfacción)...")
        cat_results_primera_pasada = [analyzer_primera_pasada.categorizar(texto) for texto in tqdm(df_con_comentarios['razon_recomendar_limpio'], desc="Categorizando (Paso 1/2)")]
        df_con_comentarios[['cat_principal', 'cat_secundaria']] = cat_results_primera_pasada

        # Paso 3: Ejecutar la segunda pasada (solo para los 'no_clasificado').
        if satisfaction_keywords:
            mask_no_clasificados = df_con_comentarios['cat_principal'] == 'no_clasificado'
            
            if mask_no_clasificados.any():
                logger.info(f"Ejecutando segunda pasada sobre {mask_no_clasificados.sum()} registros no clasificados...")

                # Crea una configuración mínima solo para la categoría de satisfacción.
                config_segunda_pasada = {
                    'categorias': {nombre_categoria_extra: satisfaction_keywords},
                    'pesos_categorias': {nombre_categoria_extra: satisfaction_weight or 1.0},
                    'umbral_secundario': config['umbral_secundario'],
                    'umbral_score_categoria': config.get('umbral_score_categoria', 0.5),
                    # Es crucial pasar los recursos lingüísticos para que la limpieza funcione correctamente.
                    'sentimiento_params': config['sentimiento_params'],
                    'stopwords': config['stopwords']
                }
                analyzer_segunda_pasada = EnhancedSentimentAnalyzer(config_segunda_pasada)
                
                # Aplica la categorización solo al subconjunto de 'no_clasificados'.
                textos_a_recategorizar = df_con_comentarios.loc[mask_no_clasificados, 'razon_recomendar_limpio']
                recat_results = [analyzer_segunda_pasada.categorizar(texto) for texto in tqdm(textos_a_recategorizar, desc="Categorizando (Paso 2/2)")]
                
                # Actualiza las columnas de categoría en el DataFrame original para las filas correspondientes.
                df_con_comentarios.loc[mask_no_clasificados, ['cat_principal', 'cat_secundaria']] = recat_results
        else:
            logger.warning(f"La categoría '{nombre_categoria_extra}' no fue encontrada. Se omitirá la segunda pasada.")

        # El análisis de sentimiento se ejecuta después de que toda la categorización está completa,
        # usando la configuración original completa.
        analyzer_sentimiento_final = EnhancedSentimentAnalyzer(config)
        prob_recomendar = df_con_comentarios.get('prob_recomendar', pd.Series([None] * len(df_con_comentarios), index=df_con_comentarios.index))
        
        sentiment_results = [analyzer_sentimiento_final.analizar_sentimiento(texto_original, prob) for texto_original, prob in tqdm(zip(df_con_comentarios['razon_recomendar'], prob_recomendar), total=len(df_con_comentarios), desc="Analizando Sentimiento")]
        sentiment_df = pd.DataFrame(sentiment_results, index=df_con_comentarios.index)
        df_con_comentarios = pd.concat([df_con_comentarios, sentiment_df], axis=1)

    # --- FIN DE LA MODIFICACIÓN ---

    # Procesa los registros sin comentarios por separado.
    if not df_sin_comentarios.empty:
        logger.info(f"Procesando {len(df_sin_comentarios)} registros sin comentarios...")
        df_sin_comentarios['cat_principal'] = 'sin_respuesta'
        df_sin_comentarios['cat_secundaria'] = 'sin_respuesta'
        
        # Aunque no hay texto, se calcula el sentimiento basado en el NPS.
        analyzer_final_sin_comentarios = EnhancedSentimentAnalyzer(config)
        prob_recomendar = df_sin_comentarios.get('prob_recomendar', pd.Series([None] * len(df_sin_comentarios), index=df_sin_comentarios.index))
        sentiment_results_empty = [analyzer_final_sin_comentarios.analizar_sentimiento(None, prob) for prob in prob_recomendar]
        sentiment_df_empty = pd.DataFrame(sentiment_results_empty, index=df_sin_comentarios.index)
        df_sin_comentarios = pd.concat([df_sin_comentarios, sentiment_df_empty], axis=1)

    # Vuelve a unir los DataFrames y los ordena por el índice original.
    df_resultado = pd.concat([df_con_comentarios, df_sin_comentarios]).sort_index()
    return df_resultado

def main(archivo_datos_path: str, ruta_categorias_json: str, ruta_sentimientos_json: str, config_externa: Dict = None):
    """
    Función principal que orquesta todo el proceso de análisis.

    Carga los datos, los pre-procesa, ejecuta el análisis de categorización y
    sentimiento, realiza un post-procesamiento (como la creación de la columna 'PERIODO')
    y guarda los resultados en un nuevo archivo CSV.

    Args:
        archivo_datos_path (str): Ruta al archivo CSV de datos de entrada.
        ruta_categorias_json (str): Ruta al archivo JSON de configuración de categorías.
        ruta_sentimientos_json (str): Ruta al archivo JSON de configuración de sentimientos.
        config_externa (Dict, optional): Permite pasar una configuración ya cargada.
                                         Defaults to None.
    """
    try:
        # Mapeo para estandarizar los nombres de las columnas del archivo de entrada.
        COLUMN_MAPPING = {
            'Fecha de Respuesta': 'fecha_respuesta',
            'rsp_id': 'respuesta_ID', 
            'rspndnt_id': 'respondent_ID', 
            'prob_rec': 'prob_recomendar',
            'comentario': 'razon_recomendar' 
        }
        # Define el orden deseado de las columnas en el archivo de salida.
        FINAL_COLUMN_ORDER = [
            'fecha_respuesta', 'respuesta_ID', 'respondent_ID', 'prob_recomendar', 
            'razon_recomendar', 'segmento_nps', 'razon_recomendar_original',
            'razon_recomendar_limpio', 'cat_principal', 'cat_secundaria', 
            'sentiment_override', 'sentiment_libre', 'puntuacion', 'PERIODO'
        ]

        logger.info(f"Cargando datos desde: {archivo_datos_path}...")
        df = pd.read_csv(archivo_datos_path)
        logger.info(f"Datos cargados. Shape: {df.shape}")

        df.rename(columns=lambda c: COLUMN_MAPPING.get(c, c), inplace=True)
        # Asegura que la columna de comentarios exista, duplicándola si es necesario para mantener el original.
        if 'razon_recomendar' in df.columns and 'razon_recomendar_original' not in df.columns:
            df['razon_recomendar_original'] = df['razon_recomendar']
        elif 'razon_recomendar' not in df.columns and 'razon_recomendar_original' in df.columns:
            df['razon_recomendar'] = df['razon_recomendar_original']
        elif 'razon_recomendar' not in df.columns:
            raise ValueError("Se requiere la columna 'razon_recomendar' o 'razon_recomendar_original'.")

        logger.info("Cargando configuración...")
        config = config_externa or cargar_config_mejorada(ruta_categorias_json, ruta_sentimientos_json)

        logger.info("Procesando datos (categorización y sentimiento)...")
        df_procesado = procesar_dataframe_eficiente(df.copy(), config)

        # Post-procesamiento: Creación de la columna 'PERIODO'.
        if 'fecha_respuesta' in df_procesado.columns:
            # Convierte la columna a string para un manejo robusto de formatos de fecha inconsistentes.
            df_procesado['fecha_respuesta_str'] = df_procesado['fecha_respuesta'].astype(str)
            # Extrae solo la parte de la fecha (YYYY-MM-DD).
            df_procesado['solo_fecha'] = df_procesado['fecha_respuesta_str'].str[:10]
            # Convierte la fecha extraída a un objeto datetime.
            df_procesado['fecha_respuesta'] = pd.to_datetime(df_procesado['solo_fecha'], errors='coerce')
            # Formatea la fecha para crear el periodo (YYYY-MM).
            df_procesado['PERIODO'] = df_procesado['fecha_respuesta'].dt.strftime('%Y-%m')
            # Elimina columnas intermedias.
            df_procesado = df_procesado.drop(columns=['fecha_respuesta_str', 'solo_fecha'])
        else:
            df_procesado['PERIODO'] = None

        # Reordena las columnas según la especificación y verifica si falta alguna.
        final_columns_present = [col for col in FINAL_COLUMN_ORDER if col in df_procesado.columns]
        df_final = df_procesado.reindex(columns=final_columns_present)
        missing_final_cols = set(FINAL_COLUMN_ORDER) - set(df_final.columns)
        if missing_final_cols:
            logger.warning(f"Columnas finales esperadas no generadas o no presentes: {missing_final_cols}")

        # Define la ruta y el nombre del archivo de salida.
        ruta_principal = os.path.dirname(os.path.dirname(archivo_datos_path))
        destino_output = os.path.join(ruta_principal, 'bases_categorizadas')
        base_categorizada_nombre = os.path.splitext(os.path.basename(archivo_datos_path))[0] + "_categorizada_v2.csv"
        ruta_salida = os.path.join(destino_output, base_categorizada_nombre)
        
        logger.info(f"Guardando resultados en: {ruta_salida}...")
        os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)
        df_final.to_csv(ruta_salida, index=False, encoding='utf-8-sig')

        logger.info(f"Proceso completado. Shape del resultado: {df_final.shape}.")
        print("¡Proceso finalizado exitosamente!")
        return df_final, ruta_salida
    except Exception as e:
        logger.error(f"Error inesperado en la ejecución de main: {str(e)}", exc_info=True)
        raise

# --- Bloque de Ejecución Principal ---
if __name__ == "__main__":
    # Este bloque se ejecuta solo cuando el script es llamado directamente.
    
    # ---- Configuración de Rutas (ajustar según sea necesario) ----
    try:
        # Define las rutas base para los datos y diccionarios de configuración.
        ruta_base_proyecto = r'RUTA_ARCHIVOS' 
        ruta_fuente_datos = os.path.join(ruta_base_proyecto, 'DIRECTORIO_PROYECTO')
        nombre_archivo_respuestas = 'base_a_procesar.csv' 
        ruta_diccionarios = os.path.join(ruta_base_proyecto, 'diccionarios')
        nombre_archivo_categorias = 'categorias_v2.json'
        nombre_archivo_sentimientos = 'sentimientos_v2.json'

        # Construye las rutas completas a los archivos.
        path_datos = os.path.join(ruta_fuente_datos, nombre_archivo_respuestas)
        path_categorias = os.path.join(ruta_diccionarios, nombre_archivo_categorias)
        path_sentimientos = os.path.join(ruta_diccionarios, nombre_archivo_sentimientos)

        # Verifica que los archivos de configuración existan antes de continuar.
        if not os.path.exists(path_categorias) or not os.path.exists(path_sentimientos):
            logger.error("Error crítico: Los archivos de configuración JSON no se encontraron en las rutas especificadas.")
            logger.error(f"Ruta de categorías buscada: {path_categorias}")
            logger.error(f"Ruta de sentimientos buscada: {path_sentimientos}")
        else:
            # Llama a la función principal para iniciar el proceso.
            df_resultado_final, ruta_de_salida_obtenida = main(
                archivo_datos_path=path_datos,
                ruta_categorias_json=path_categorias,
                ruta_sentimientos_json=path_sentimientos
            )
            # Imprime una muestra del resultado y la ruta del archivo guardado.
            if df_resultado_final is not None:
                print("\nPrimeras 5 filas del resultado:")
                print(df_resultado_final.head())
            print(f"\nEl archivo de resultados fue guardado en: {ruta_de_salida_obtenida}")

    except Exception as e:
        print(f"El proceso principal falló: {e}")

Código para ver la distribución de las respuestas por sentiment y categoría.

In [ ]:
dff = df_resultado_final.copy()

def resumen_sentiment_por_categoria(
    df: pd.DataFrame,
    *,
    cat_col: str  = "cat_principal",
    sent_col: str = "sentiment_override"
) -> pd.DataFrame:
    # Pivot counts
    conteos = df.groupby([cat_col, sent_col]).size().unstack(fill_value=0)
    # Aseguro columnas
    for nivel in ("positivo","neutral","negativo"):
        if nivel not in conteos:
            conteos[nivel] = 0
    conteos["Total"] = conteos[["positivo","neutral","negativo"]].sum(axis=1)
    # Porcentajes
    conteos["pct_positivo"] = (conteos["positivo"]/conteos["Total"]*100).round(0).astype(int)
    conteos["pct_neutral"]  = (conteos["neutral"]/conteos["Total"]*100).round(0).astype(int)
    conteos["pct_negativo"] = (conteos["negativo"]/conteos["Total"]*100).round(0).astype(int)
    # Ajusto nombres
    return (
        conteos
        .reset_index()
        .rename(columns={cat_col:"Categoria"})
        [["Categoria","positivo","neutral","negativo","Total",
          "pct_positivo","pct_neutral","pct_negativo"]]
    )

# Genero la tabla y devuelvo:
tabla_sent = resumen_sentiment_por_categoria(dff)
tabla_sent.to_csv(
    fr'TU_RUTA',
    index=False
)
tabla_sent

Código para ver la distribución de las respuestas entre las categorías principales.

In [ ]:
import pandas as pd
df_procesado = pd.read_csv(
    f"{ruta_de_salida_obtenida}"
)
# Obtener el conteo de categorías
conteo_categorias = df_procesado.cat_principal.value_counts()

# Calcular el porcentaje que representa cada categoría
total_registros = len(df_procesado)
porcentaje_categorias = df_procesado.cat_principal.value_counts(normalize=True) * 100

# Crear un DataFrame para mostrar ambos valores
resultado = pd.DataFrame({
    'Conteo': conteo_categorias,
    'Porcentaje (%)': porcentaje_categorias.round(2)
})

# Mostrar el resultado
print(f"Total de registros: {total_registros}")
resultado
